In [2]:
%load_ext autoreload
%config Completer.use_jedi = False

In [3]:
%autoreload 2
import FEMOL
import numpy as np
np.set_printoptions(linewidth=200)
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt


# Development Notebook for the FEMOL Project

In [59]:
class Element(object):
    
    N_nodes = 0
    
    @staticmethod
    def shape(xi, eta):
        return xi -eta
    
    def __init__(self, points, Ndof):
        self.x, self.y = points.transpose()[:2]
        self.N_dof = Ndof
        self.size = Ndof*self.N_nodes
        
    def shape(self, x, y):
        return x - self.x[0] + y - self.y[0]
    
    def Ke(self):
        return np.array([[self.x[0], 0],[0, self.y[0]]])
        

In [60]:
class T2(Element):
    
    N_nodes = 2
    
    @staticmethod
    def shape(xi, eta):
        return xi + eta
    
    def __init__(self, points, Ndof):
        super().__init__(points, Ndof)
        
    def shapexy(self, x, y):
        return (self.x[0] + x) * (self.y[0] + y)

In [19]:
# Circle mesh with R = 0.5 and ~25 ** 2 elements
R = 0.5  # m
N_ele = 10
mesh = FEMOL.mesh.circle_Q4(R, N_ele)

# Problem definition
thickness = 0.005
aluminium = FEMOL.materials.IsotropicMaterial(71e9, 0.33, 2700)

# Create a FEM Problem from the mesh (compute displacement with a plate bending model)
problem1 = FEMOL.FEM_Problem(mesh=mesh, physics='modal', model='plane')
problem2 = FEMOL.FEM_Problem(mesh=mesh, physics='modal', model='plane')

problem1.define_materials(aluminium, aluminium)
problem1.define_tensors(thickness/2, thickness/2)

problem2.define_materials(aluminium)
problem2.define_tensors(thickness)

circle_domain = FEMOL.domains.outside_circle(0, 0, R - 0.005)

# Fix all the degrees of freedom
problem1.add_fixed_domain(circle_domain)
problem2.add_fixed_domain(circle_domain)

# Solve the eigenvalue problem and store the frequencies
w1, _ = problem1.solve(filtre=2)
w2, _ = problem2.solve(filtre=2)

print(np.allclose(w1[:10], w2[:10]))
print(np.allclose(problem1.M.toarray(), problem2.M.toarray()))

solving using scipy
solved in :  0.21208500862121582  s
solving using scipy
solved in :  0.1258230209350586  s
True
True
